In [21]:
!java -version
# C:\Program Files\Java\jre1.8.0_321

'java'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.


In [1]:
import csv
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

mc = ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max', save_best_only=True)   # validation의 정확도 최대일때 저장
es = EarlyStopping(monitor='val_accuracy',mode='max',patience=5)
# tvalidation의 정확도 최대일때. patience는 기준되는 값이 연속으로 몇 번 이상 향상되지 않을 때 종료시킬 것인지

In [2]:
df = pd.read_csv('C:/git_project/natural/data/okt/X_train.csv')
df

,0,1,2,3,4,5,6,7,8,9,...,35,36,37,38,39,40,41,42,43,44
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,72,528,32,408,699
1,0,0,0,0,0,0,0,0,0,0,...,507,45,674,2,101,1360,36,1003,720,35
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,19,577,4,21,8,234,3
3,0,0,0,0,0,0,0,0,0,0,...,0,6396,125,5999,253,70,12,14,49,3566
4,0,0,0,0,0,0,0,0,0,0,...,1122,257,3,10090,11825,7,1116,1,1404,257
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155863,0,0,0,0,0,1087,4173,5,7939,8,...,15,40,73,182,39,444,66,10,266,596
155864,0,0,0,0,0,0,0,0,0,0,...,146,6,819,1907,659,15,3,692,4854,35
155865,0,0,0,0,0,0,0,0,0,0,...,105,191,105,191,95,45,61,1116,865,636
155866,0,0,0,0,0,0,0,0,0,0,...,5,285,1116,1610,41,5,1461,1116,93,3


In [3]:
X_train_data = pd.read_csv('C:/git_project/natural/data/okt/X_train.csv')   # 데이터 불러오기
X_test_data = pd.read_csv('C:/git_project/natural/data/okt/X_test.csv')
y_train_data = pd.read_csv('C:/git_project/natural/data/okt/y_train.csv')   # 데이터 불러오기
y_test_data = pd.read_csv('C:/git_project/natural/data/okt/y_test.csv')

In [4]:
X_train_data.dropna(how='any', inplace=True)    # null값 탈락
X_test_data.dropna(how='any', inplace=True)
y_train_data.dropna(how='any', inplace=True)
y_test_data.dropna(how='any', inplace=True)

In [5]:
train_X = np.array(X_train_data)    # array로 변경하여 tensor
test_X = np.array(X_test_data)
train_X, test_X

(array([[   0,    0,    0, ...,   32,  408,  699],
        [   0,    0,    0, ..., 1003,  720,   35],
        [   0,    0,    0, ...,    8,  234,    3],
        ...,
        [   0,    0,    0, ..., 1116,  865,  636],
        [   0,    0,    0, ..., 1116,   93,    3],
        [   0,    0,    0, ...,   95,  154,    4]], dtype=int64),
 array([[   0,    0,    0, ...,    0,  815,  119],
        [   0,    0,    0, ...,   64,  745,   34],
        [   0,    0,    0, ...,   86,  170,  295],
        ...,
        [   0,    0,    0, ...,  180, 3045,   40],
        [   0,    0,    0, ...,   11,  255,  151],
        [   0,    0,    0, ...,  143,   59,   46]], dtype=int64))

In [6]:
train_y = np.array(y_train_data)
test_y = np.array(y_test_data)
train_y, test_y

(array([[0],
        [1],
        [0],
        ...,
        [1],
        [1],
        [1]], dtype=int64),
 array([[1],
        [0],
        [0],
        ...,
        [0],
        [0],
        [0]], dtype=int64))

# Model Building

In [7]:
vocab_size = 19395                                                   #전처리 이후 사이즈 확인

model = keras.Sequential()
model.add(keras.layers.Embedding(vocab_size, 32, mask_zero=True))   # 임베딩 32
model.add(keras.layers.Dropout(0.5))                                 # dropout 0.2 - 0.5사이
model.add(keras.layers.Bidirectional(keras.layers.GRU(64)))          # 노드  64
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(16, activation='softsign'))            # softsign, tanh로 돌려봄
model.add(keras.layers.Dropout(0.5))  
model.add(keras.layers.Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          620640    
                                                                 
 dropout (Dropout)           (None, None, 32)          0         
                                                                 
 bidirectional (Bidirectiona  (None, 128)              37632     
 l)                                                              
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense (Dense)               (None, 16)                2064      
                                                                 
 dropout_2 (Dropout)         (None, 16)                0         
                                                        

In [8]:
model.compile(optimizer=keras.optimizers.Adam(0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Training

In [9]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train_data, y_train_data))
val_dataset = tf.data.Dataset.from_tensor_slices((X_test_data, y_test_data))

train_dataset = train_dataset.shuffle(buffer_size=1024, reshuffle_each_iteration=True).batch(128)  # batch_size 64,128,256 확인
val_dataset = val_dataset.batch(1000)

train_dataset = train_dataset.prefetch(4)

In [10]:
history = model.fit(train_dataset, epochs=10, validation_data=val_dataset, verbose=1,callbacks=[mc]) 
#callbacks로 체크포인트 확인 epochs 10,20회 반복으로 시도

Epoch 1/10
1218/1218 [==============================] - 80s 61ms/step - loss: 0.4218 - accuracy: 0.8080 - val_loss: 0.5001 - val_accuracy: 0.7488
Epoch 2/10
1218/1218 [==============================] - 76s 62ms/step - loss: 0.3565 - accuracy: 0.8517 - val_loss: 0.4320 - val_accuracy: 0.7949
Epoch 3/10
1218/1218 [==============================] - 78s 64ms/step - loss: 0.3388 - accuracy: 0.8620 - val_loss: 0.3942 - val_accuracy: 0.8234
Epoch 4/10
1218/1218 [==============================] - 75s 61ms/step - loss: 0.3217 - accuracy: 0.8702 - val_loss: 0.3829 - val_accuracy: 0.8260
Epoch 5/10
1218/1218 [==============================] - 110s 91ms/step - loss: 0.3093 - accuracy: 0.8767 - val_loss: 0.3809 - val_accuracy: 0.8265
Epoch 6/10
1218/1218 [==============================] - 122s 100ms/step - loss: 0.2974 - accuracy: 0.8817 - val_loss: 0.3640 - val_accuracy: 0.8356
Epoch 7/10
1218/1218 [==============================] - 115s 94ms/step - loss: 0.2854 - accuracy: 0.8877 - val_loss: 0.38

In [34]:
from keras.models import load_model
model.save('GRU_okt_model.h5')

In [35]:
loaded_model = load_model('GRU_okt_model.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test_data, y_test_data)[1]))

1527/1527 [==============================] - 11s 6ms/step - loss: 0.3589 - accuracy: 0.8474

 테스트 정확도: 0.8474


### 하이퍼 파라미터 조절
#### OKT, tanh 사용 epochs 10회씩
drop out ---> 0.2 
batch size --->   256 =>128
layer ---->  32 , 중간 GRU(64)는 건들지 않음,  16


drop out --->    0.4
batch size ---> 128
layer ---->  32  , 중간 GRU(64)는 건들지 않음,  16


drop out --->  0.4 
batch size --->  128
layer ---->  32 =>16 , 중간 GRU(64)는 건들지 않음,  16=>8


drop out --->   0.5
batch size --->   128
layer ---->  16 , 중간 GRU(64)는 건들지 않음,  8


drop out --->   0.5
batch size --->   128
layer ---->  8 , 중간 GRU(64)는 건들지 않음,  4


drop out --->   0.5
batch size --->   64
layer ---->  8 , 중간 GRU(64)는 건들지 않음,  4


drop out --->   0.5
batch size --->   128
layer ---->  32 , 중간 GRU(64)는 건들지 않음,  16    =======> 최종 하이퍼파라미터

### 이후 
최종 하이퍼 파라미터 사용

OKT

softsign과 tanh 비교

### 최고 수치
# tanh         val_accuracy:0.8450,  val_loss:0.3586
# softsign   val_accuracy: 0.8474,  val_loss:0.3589

In [32]:
from tensorflow.keras.models import load_model
import random
from konlpy.tag import Okt
import os
from tensorflow.keras.preprocessing.text import Tokenizer
import pickle
from tensorflow.keras.preprocessing.sequence import pad_sequences

okt = Okt()
tokenizer  = Tokenizer()
loaded_model = model
max_len = 42

# DATA_CONFIGS = 'data_configs.json'
# prepro_configs = json.load(open(DATA_PATH+DATA_CONFIGS,'r'))
word_vocab = tokenizer.word_index
model['vocab'] = word_vocab
tokenizer.fit_on_texts(word_vocab)

global corpus
global avg_emo
global score

corpus = []
avg_emo = 0


heal = ['여수 밤바다', '파주 평화누리공원', '순천 갈대밭', '제주도 영실코스', 
         '진주 진양호', '장흥유원지', '대구 앞산정만대', '춘천 해피초원농장' ,' 속초 해수욕장', 
         '경주 불국사', '포항 호미곶', '남해 두모마을', '대관령 하늘목장', '군산 철길마을', 
         '국립 광릉수목원', '금선사 템플스테이', '보성 제암산자연휴양림','군산 선유도']
 
extreme= ['통영 어드벤처 타워', '제주도  스쿠버다이빙', '단양 패러글라이딩', 
          '강원 내린천 래프팅', '충주 스카이다이빙', '하남 스포츠몬스터', '서울 한강 워터 제트팩', 
          '일산 인공 서핑', '영월 동강래프팅', '여수 스카이플라이', '문경 패러글라이딩', 
          '경남 하동 하동알프스레포츠', '인천 스카이 짚라인', '강화 루지']
 
movie=['루카','콰이어트 플레이스','크루엘라','컨저링 3:악마가 시켰다','여고괴담 여섯번째 이야기','분노의 질주: 더 얼티메이트','캐시트럭',
'클라이밍',
'그 여름, 가장 차가웠던',
'폭력의 그림자',
'청춘 선거',
'그레타 툰베리',
'낫아웃',
'마세티 킬즈',
'프로페서 앤 매드맨',
'화이트 온 화이트',
'아야와 마녀',
'까치발',
'플래시백',
'애플',
'혼자 사는 사람들',
'강호아녀',
'파이프라인',
'분노의 질주']
 
#음악 (네이버 바이브 참고 1위~20위)
 
k_balad=['Timeless-SG워너비',
'추적이는 여름 비가 되어-장범준',
'밤하늘의 별을 (2020)-경서',
'어떻게 이별까지 사랑하겠어, 널 사랑하는 거지-AKMU (악동뮤지션)',
'내 입술 따뜻한 커피처럼-청하 Colde(콜드)',
'서울의 잠 못 이루는 밤 (Feat. 이수현)-10CM',
'잠이 오질 않네요-장범준',
'I Love U-성시경',
'내사람 (Partner For Life)-SG워너비',
'취기를 빌려-산들',
'안녕 (Hello)-조이',
'밤편지-아이유(IU)',
'Anti-Romantic-투모로우바이투게더',
'너의 모든 순간-성시경',
'좋을텐데 (If Only) (Feat. 폴킴)-조이',
'봄 안녕 봄-아이유(IU)',
'Love poem -아이유(IU)',
'아이와 나의 바다-아이유(IU)',
'벌써 일년-반하나& MJ(써니사이드)',
'이렇게 좋아해 본 적이 없어요 -CHEEZE (치즈)'
]
 
k_dance=[
'Butter-방탄소년단',
'Next Level-aespa',
'Dun Dun Dance-오마이걸(OH MY GIRL)',
'치맛바람 (Chi Mat Ba Ram)-브레이브걸스(Brave Girls)',
'Alcohol-Free-TWICE(트와이스)',
"롤린 (Rollin')-브레이브걸스(Brave Girls)",
'라일락-아이유(IU)',
'ASAP-STAYC(스테이씨)',
'Dynamite-방탄소년단',
'상상더하기-MSG워너비',
'Celebrity-아이유(IU)',
'상상더하기-라붐(LABOUM)',
'Ready to love-세븐틴',
'Dolphin-오마이걸(OH MY GIRL)',
'Lovesick Girls-BLACKPINK',
'Pool Party (Feat. 이찬 of DKB)-브레이브걸스(Brave Girls)',
"Heaven's Cloud-세븐틴",
'체념-정상동기(김정수, 정기석, 이동휘, 이상이)',
'어푸 (Ah puh)-아이유(IU)',
]
 
k_hip=[
'마.피.아. In the morning-ITZY(있지)',
'봄날-방탄소년단',
'Life Goes On-방탄소년단',
'맛 (Hot Sauce)-NCT DREAM',
'밸런스 게임-투모로우바이투게더',
'GAM3 BO1-세븐틴',
'비도 오고 그래서 (Feat. 신용재)-헤이즈 (Heize)',
'METEOR-창모(CHANGMO)',
'DNA-방탄소년단',
'IDOL-방탄소년단',
'FAKE LOVE-방탄소년단',
'피 땀 눈물-방탄소년단',
'사이렌-호미들',
'멜로디-ASH ISLAND',
'I NEED U-방탄소년단',
'아무노래-지코 (ZICO)',
'어떻게 지내 (Prod. By VAN.C)-오반(OVAN)',
'Rainy day (Feat. ASH ISLAND, Skinny Brown)-PATEKO(파테코)',
'뚜두뚜두 (DDU-DU DDU-DU)-BLACKPINK']
 
 
trt=[
'이제 나만 믿어요-임영웅',
'별빛 같은 나의 사랑아-임영웅',
'다시 사랑한다면 (김필 Ver.)-임영웅',
'HERO-임영웅',
'미워요-임영웅',
'잊어야 한다는 마음으로-임영웅',
'계단말고 엘리베이터-임영웅',
'소나기-임영웅',
'바보같지만-임영웅',
'따라따라-임영웅',
'당신-임영웅',
'내 마음 별과 같이-임영웅',
'고맙소-김호중',
'만개 (Prod. 신지후)-김호중',
'나보다 더 사랑해요-김호중',
'애인이 되어줄게요 (Prod. 알고보니, 혼수상태)-김호중',
'퇴근길-김호중',
'할무니-김호중',
'우산이 없어요-김호중',
'천년의 사랑-김호중'
]
 
 
f_dance=[
'You-Regard, Troye Sivan, Tate McRae',
'Closer (Feat. Halsey)-The Chainsmokers',
'Faded-Alan Walker',
'One Kiss-Calvin Harris, Dua Lipa',
'Heartbreak Anthem-Galantis, David Guetta, Little Mix',
'Something Just Like This-The Chainsmokers, Coldplay',
'This Is What You Came For (Feat. Rihanna)-Calvin Harris',
'The Middle-Zedd, Grey, Maren Morris',
'Symphony (Feat. Zara Larsson)-Clean Bandit',
'Wake Me Up-Avicii',
'Waste It On Me (Feat. BTS(방탄소년단))-Steve Aoki',
'How To Love (Feat. Sofia Reyes)-Cash Cash',
'Bad Boy (with Wiz Khalifa, bbno$, MAX)-Yung Bae, Wiz Khalifa, bbno$, MAX',
'Titans (Feat. Sia & Labrinth) (Imanbek Remix)-Major Lazer',
'Feels (Feat. Pharrell Williams, Katy Perry, Big Sean)-Calvin Harris',
'Rise (Feat. Jack & Jack)-Jonas Blue',
'Mama (Feat. William Singe)-Jonas Blue',
'Just Got Paid (Feat. French Montana)-Sigala, Ella Eyre, Meghan Trainor',
'Love Line-Shift K3y, Tinashe',
'Lonely Together (Feat. Rita Ora)-Avicii'
]
 
newage=[
'River Flows In You-이루마',
'Letter From The Earth (지구에서 온 편지)-김광민',
'익숙한 그 집 앞-유희열',
'처음부터 지금까지 (Inst.)-박정원',
'냉정과 열정 사이 OST (冷靜と情熱のあいだ)-Ryo Yoshimata',
'''Tomorrow's Promise-Kevin Kern''',
"Mia & Sebastian's Theme-Justin Hurwitz",
'Recuerods de la Alhambra (알함브라 궁전의 추억)-Claude Ciari',
'''Gabriel's Oboe-Ennio Morricone''',
'Rain-Ryuichi Sakamoto',
'Romance-Yuhki Kuramoto',
'Second Romance-Yuhki Kuramoto',
'Crystal Rainbow-데이드림(Daydream)',
'My Road (Live)-Lee Oskar',
'Last Carnival-Acoustic Cafe',
'Return To The Heart-David Lanz',
'Adagio-Secret Garden',
'Loving You-Kenny G'
]

korean = ["삼계탕", "삼겹살", "곱창", "찜닭", "오리고기", "소고기", 
          "국밥", "닭도리탕", "낙곱새", "라면", "비빔밥", "칼국수", 
          "수제비", "갈비", "제육볶음"]

western = ["스테이크", "파스타", "필라프", "감바스", "리조또", "샐러드", 
           "피자", "빠에야", "플래터", "스튜"]

asian = ["짜장면", "뿌팟퐁커리", "팟타이", "나시고랭", "쌀국수", "미고랭",
         "카레", "마라탕", "마라샹궈", "훠궈", "돈까스", "월남쌈", "라멘", 
         "탄탄멘", "규동", "꿔바로우", "똠양꿍", "물냉면"]

spicy = ["떡볶이", "김치찜", "김치찌개", "감자탕", "짬뽕", "닭발", "부대찌개",
         "순두부찌개", "아구찜", "해물찜", "육개장", "낙지볶음", "쭈꾸미", 
         "돼지갈비찜", "소꼬리찜", "비빔냉면"]

dessert = ["와플", "마카롱", "빙수", "크로크모슈", "케이크", "허니바게트볼",
           "머쉬룸수프볼", "에그데니쉬", "케이크", "치아바타", "호두파운드케이크",
           "쿠키", "허니브레드", "오믈렛", "베이글"]

snack = ["닭강정", "양꼬치", "핫윙", "소떡소떡", "가라아게", "콘치즈", "감자튀김", 
         "치킨너겟", "치킨", "낫쵸", "소시지", "버터구이", "계란찜", "핫도그", 
         "해쉬브라운"]

coffee = ["아메리카노", "콜드브루", "바닐라 라떼", "카페 라떼", "카라멜 마키아또",
          "카페 모카", "바닐라 프라페", "카페모카 프라페", "연유 라떼", "화이트 모카",
          "민트 모카", "헤이즐넛 라떼", "에스프레소", "오곡 프라페", "쿠앤크 프라페"]

beverage = ["초코 라떼", "민트초코 라떼", "밀크티", "흑당 버블티", "레몬차", "자몽차",
            "유자차", "모히토", "요거트 스무디", "블루베리 스무디", "딸기 스무디", "애플망고 스무디",
            "레몬 에이드", "자몽 에이드", "생과일 주스"]

motivation = ["자신을 믿어라. 자신의 능력을 신뢰하라. 겸손하지만 합리적인 자신감 없이는 성공할 수도 행복할 수도 없다. - 노먼 빈센트 필",
"조금 더 많이 인내하자. 조금 더 많이 노력하자. 그러면 절망적 실패로 보였던 것이 빛나는 성공으로 변할 수 있다. - 알버트 휴버드",
"당신이 인생의 주인공이기 때문이다. 그 사실을 잊지말라. 지금까지 당신이 만들어온 의식적 그리고 무의식적 선택으로 인해 지금의 당신이 있는것이다. - 바바라 홀",
"먹는 칼로리보다 에너지 소모가 적으면 살이 찌듯이, 걱정만 하고 행동하지 않으면 걱정이 찐다.",
"이미 끝나버린 일을 후회하기 보다는 하고 싶었던 일을 하지 못한 것을 후회하라 - 탈무드",
"기회가 주어지면 최선을 다하는 것이 아니라 최선을 다하고 있으면 기회가 주어지는 것이다 - 신영준",
"낭비한 시간에 대한 후회는 더 큰 시간 낭비이다 - 메이슨 쿨리",
"성공은 매일 부단하게 반복된 작은 노력의 합산이다.",
"현명한 사람은 앉아서 손해 본 것을 한탄만 하지 않고 즐겁게 그 손해를 회복할 방법을 찾는다. - 셰익스피어",
"고통을 주지 않는것은 쾌락도 주지 않는다 - 몽테뉴",
"시간은 간다",
"살아가는 사람들 중 대부분은 자신에게 올 기회를 기다리나 기회라는 것은 기다리는 사람에게는 쉽게 오지 않는 법이다",
"기회를 얻을 수 있게 기다리는 사람이 되기보다는 기회를 얻을 수 있는 실력을 먼저 쌓아야 한다. 자신이 하는 일에 열중하고 노력하다보면 자연스럽게 기회는 찾아온다.",
"변화를 위해서 가장 중요한 것은 행동하는 첫걸음이다.",
"무엇이든 하루아침에 만들어지는 것은 없다. 로마 또한 하루아침에 만들어지지 않았다. 이 말은 무언가를 만들기 위해서는 그것을 만들기 위해 노력하고 집중 해야 한다는 것이다.",
"스스로를 믿고 자신이 가지고 있는 능력을 신뢰해야 한다. 하지만 거만하게 행동하지 말고 겸손해라. 성공을 위해서 자신감이 필요하지만 오만함은 필요하지 않다.",
             "끝난 일은 언급할 필요가 없으며 지난 일은 허물을 물을 필요가 없다. - 공자",
"어렵고 힘든 상황일수록 서두르지 말고 침착해라. 성급하게 하는 행동에는 실수가 포함되기 쉽다.",
"나의 하루를 설명할 수 있는 사람이 곁에 있다는 건 생각보다 기분 좋은 일이야 그러니 너도 생각보다 좋은 사람이지 - 흔글",
"잠 못 자고 있지, 얼른 자, 걱정하는 일 안 생겨 좋은 일은 아니더라도 아무 일 없을 거야 혼자 있는 새벽을 걱정으로 보내지는 마 - 흔글",
"봄바람도 살랑살랑 불고 꽃도 예쁘게 피어있으니 얼마나 놀고 싶겠냐만은, 그래도 그 시간들을 이겨내면 너의 인생에 꽃이 필 테니 조금만 참고 바람을 이겨내기를 - 흔글"]

category = [heal,extreme,movie,k_balad,k_dance,k_hip,trt,f_dance,newage,korean,western,asian,spicy,dessert,snack,coffee,beverage]
f8 = [extreme,movie,k_balad,k_dance,k_hip,trt,f_dance,korean,western,asian,spicy,dessert,snack,coffee,beverage]
f6 = [extreme,movie,k_balad,k_dance,k_hip,trt,f_dance,korean,western,asian,spicy,dessert,snack,coffee,beverage]
f3 = [heal,extreme,movie,k_balad,k_dance,k_hip,trt,f_dance,newage,korean,western,asian,spicy,dessert,snack,coffee,beverage, dessert]
f1 = [heal,movie,k_balad,k_dance,k_hip,trt,f_dance,newage,korean,western,asian,spicy,dessert,snack,coffee,beverage, dessert]
f0 = [heal,movie,k_balad,k_dance,k_hip,f_dance,newage,korean,western,asian,spicy,spicy,dessert,snack,snack,coffee,beverage]

def recomend_sys(new_sentence):
    global score
    global timecheck
    global corpus
    global score
    global avg_emo

    print(new_sentence)
    new_sentence = okt.morphs(new_sentence, stem=True) # 토큰화
    new_sentence = [word for word in new_sentence if not word in stopwords] # 불용어 제거
    encoded = tokenizer.texts_to_sequences([new_sentence]) # 정수 인코딩
    pad_new = pad_sequences(encoded, maxlen = max_len) # 패딩
    score = float(loaded_model.predict(pad_new)) # 예측
    
    if(score > 0.5):
        print("{:.2f}% 확률로 긍정 리뷰입니다.\n".format(score * 100))
    else:
        print("{:.2f}% 확률로 부정 리뷰입니다.\n".format((1 - score) * 100))
      
    corpus.append(score)
    if len(corpus) >= 5:
        avg_emo = sum(corpus)/len(corpus)
        if(avg_emo > 0.8):
            pick = random.choice(f8)
            if pick == motivation: # | pick == rest:
                print("기분 점수 : {0:.2f}".format(avg_emo*100))
                print("당신을 위한 한마디 : {1}\n".format(random.choice(pick)))
                corpus=[]
            else:
                print("기분 점수 : {0:.2f}".format(avg_emo*100))
                print("오늘처럼 좋은 날엔 {0} 어떠신가요?\n".format(random.choice(pick)))
                corpus=[]

        elif(avg_emo > 0.6): 
            pick = random.choice(f6)
            if pick == motivation: # | pick == rest:
                print("기분 점수 : {0:.2f}".format(avg_emo*100))
                print("당신을 위한 한마디 : {1}\n".format(random.choice(pick)))
                corpus=[]
            else:
                print("기분 점수 : {0:.2f}".format(avg_emo*100))
                print("좋은 일있으신가요? 오늘 {0} 어떠세요?\n".format(random.choice(pick)))
                corpus=[]
        elif(avg_emo > 0.4) :
            pick = random.choice(category)
            if pick == motivation: # | pick == rest:
                print("기분 점수 : {0:.2f}".format(avg_emo*100))
                print("당신을 위한 한마디 : {1}\n".format(random.choice(pick)))
                corpus=[]
            else:
                print("기분 점수 : {0:.2f}".format(avg_emo*100))
                print("오늘같은 날에는 {0} 어때요? 기분이 좋아질거에요!!\n".format(random.choice(pick)))
                corpus=[]
        elif(avg_emo > 0.3) :
            pick = random.choice(f3)
            if pick == motivation: # | pick == rest:
                print("기분 점수 : {0:.2f}".format(avg_emo*100))
                print("당신을 위한 한마디 : {1}\n".format(random.choice(pick)))
                corpus=[]
            else:
                print("기분 점수 : {0:.2f}".format(avg_emo*100))
                print("꿀꿀한 오늘 {0} 추천해요.\n".format(random.choice(pick)))
                corpus=[]
        elif(avg_emo > 0.18) :
            pick = random.choice(f1)
            if pick == motivation: # | pick == rest:
                print("기분 점수 : {0:.2f}".format(avg_emo*100))
                print("당신을 위한 한마디 : {1}\n".format(random.choice(pick)))
                corpus=[]
            else:
                print("기분 점수 : {0:.2f}".format(avg_emo*100))
                print("오늘 {0} 어떠세요? 안좋은 기분을 환기시켜줄 거에요.\n".format(random.choice(pick)))
                corpus=[]
        else:
            pick = random.choice(f0)
            if pick == motivation: # | pick == rest:
                print("기분 점수 : {0:.2f}".format(avg_emo*100))
                print("당신을 위한 한마디 : {1}\n".format(random.choice(pick)))
                corpus=[]
            else:
                print("기분 점수 : {0:.2f}".format(avg_emo*100))
                print("오늘 안좋은 일이 있으셨나요. 오늘같은 날 {0} 어떠세요?\n".format(random.choice(pick)))
                corpus=[]
    else:
        print('감정 분석까지 {0}개의 문장이 남았어요!'.format(5-len(corpus)))

TypeError: 'Sequential' object does not support item assignment

In [29]:
recomend_sys('나는 한화팬입니다')

나는 한화팬입니다


NameError: name 'tokenizer' is not defined